In [1]:
from __future__ import division, print_function, absolute_import
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"   
import random
import tensorflow as tf
#from tensorflow import keras as K
from tensorflow.python.framework import ops
#%matplotlib inline
#tf.enable_eager_execution()

from tensorflow.python.tools import freeze_graph

from PIL import Image
# Import MNIST data

#mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

import matplotlib.pyplot as plt
import numpy as np
import PIL.ImageOps  as ImageOps
from PIL import Image, ImageDraw
import time
import shutil

print ("done")


done


In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
print(get_available_gpus())

['/device:GPU:0']


In [3]:
#import tensorflow.compat.v1 as tff
#tff.disable_v2_behavior()
#v1=1

tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.set_memory_growth(physical_devices[1], True)
tf.config.experimental.set_memory_growth(physical_devices[2], True)



In [4]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)


config = ConfigProto(gpu_options=gpu_options)
config.gpu_options.allow_growth = True




In [5]:
import glob
import matplotlib
from matplotlib import pyplot as plt

import numpy as np
#import imageio as im



In [6]:


from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.backend import *
import tensorflow.keras.backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import *
"""
def customLoss(yTrue,yPred):
    
    loss_op=(
    #mean_squared_error(yTrue, yPred)
        logcosh(yTrue, yPred)
    )
    return loss_op
"""



def ageLoss(yTrue,yPred):
    ageYTrue=yTrue[:,0]
    ageYPred=yPred[:,0]

    
    ageYTrue_masked = tf.boolean_mask(ageYTrue, tf.not_equal(ageYTrue, -1))
    ageYPred_masked = tf.boolean_mask(ageYPred, tf.not_equal(ageYTrue, -1))
    

    
    ageLoss=tf.keras.losses.logcosh(ageYTrue_masked, ageYPred_masked)
    
 
    
    loss_op=(
       K.mean(ageLoss)
    )
    
    return loss_op


def genderLoss(yTrue,yPred):

    genderYTrue=yTrue[:,1]
    genderYPred=yPred[:,1:3]

    
    genderYTrue_masked = tf.boolean_mask(genderYTrue, tf.not_equal(genderYTrue, -1))
    genderYPred_masked = tf.boolean_mask(genderYPred, tf.not_equal(genderYTrue, -1))
    

    
    genderLoss=tf.keras.losses.sparse_categorical_crossentropy(genderYTrue_masked,genderYPred_masked)
    
 
    
    loss_op=(
       K.mean(genderLoss)*12
    )
    
    return loss_op


def emoLoss(yTrue,yPred):

    emoYTrue=yTrue[:,2]
    emoYPred=yPred[:,3:9]

    
    emoYTrue_masked = tf.boolean_mask(emoYTrue, tf.not_equal(emoYTrue, -1))
    emoYPred_masked = tf.boolean_mask(emoYPred, tf.not_equal(emoYTrue, -1))
    

    
    emoLoss=tf.keras.losses.sparse_categorical_crossentropy(emoYTrue_masked,emoYPred_masked)
    
 
    
    loss_op=(
       K.mean(emoLoss)*8
    )
    
    return loss_op

In [7]:

datapath='./dataarray/three_attributes/'
inputarraytrain=np.load(datapath+'theImageArraytrain.npy')
inputarraytest=np.load(datapath+'theImageArraytest.npy')




labelarraytrain=np.load(datapath+'theLabelArraytrain.npy')
labelarraytest=np.load(datapath+'theLabelArraytest.npy')

namearraytrain=np.load(datapath+'theNameArraytrain.npy')
namearraytest=np.load(datapath+'theNameArraytest.npy')

In [8]:
print(type(inputarraytrain[0][0][0][0]))

<class 'numpy.uint8'>


In [9]:



from tensorflow.keras import backend as K





def _relu6(x):
    """Relu 6
    """
    return K.relu(x, max_value=6.0)

def _hard_swish(x):


    return x * K.relu(x + 3.0, max_value=6.0) / 6.0

def _return_activation( x, nl):

    if nl == 'HS':
        x = Activation(_hard_swish)(x)
    if nl == 'RE':
        x = Activation(_relu6)(x)

    return x

def _conv_block(inputs, filters, kernel, strides, nl):


    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x = Conv2D(filters, kernel, padding='same', strides=strides)(inputs)
    x = BatchNormalization(axis=channel_axis)(x)

    return _return_activation(x, nl)


def _bottleneck( inputs, filters, kernel, e, s, squeeze, nl):


    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    input_shape = K.int_shape(inputs)
    tchannel = input_shape[channel_axis] * e
    r = s == 1 and input_shape[3] == filters

    x = _conv_block(inputs, tchannel, (1, 1), (1, 1), nl)

    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)

    if squeeze:
        input_channels = x.shape[-1]
        x1 = GlobalAveragePooling2D()(x)

        x1 = Dense(input_channels, activation='relu')(x1)
        x1 = Dense(input_channels, activation='hard_sigmoid')(x1)
        x1= Reshape((1,1,input_channels))(x1)

        x = Multiply()([x, x1])



    x = _return_activation(x, nl)

    x = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)

    if r:
        x = Add()([x, inputs])

    return x



In [10]:

# scalling:1 1.2 day, gendeaccu:93.7 age25peraccu=63.7 agemeandiff=8

#small
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Reshape
from tensorflow.keras.utils import plot_model




def MobileNetV3_Large(shape,n_class,scalling):

    inputs = Input(shape=shape)

    x = _conv_block(inputs, int(16*scalling), (3, 3), strides=(2, 2), nl='HS')
    
    x = _bottleneck(x, int(20*scalling), (3, 3), e=1, s=1, squeeze=True, nl='RE')
    x =  _bottleneck(x, int(24*scalling), (3, 3), e=2, s=2, squeeze=True, nl='RE')
    x =  _bottleneck(x, int(24*scalling), (3, 3), e=3, s=1, squeeze=False, nl='RE')
    x =  _bottleneck(x, int(40*scalling), (5, 5), e=4, s=2, squeeze=True, nl='HS')
    x =  _bottleneck(x, int(40*scalling), (5, 5), e=6, s=1, squeeze=True, nl='HS')
    x =  _bottleneck(x, int(40*scalling), (5, 5), e=6, s=1, squeeze=True, nl='HS')
    x =  _bottleneck(x, int(48*scalling), (5, 5), e=3, s=1, squeeze=True, nl='HS')
    x =  _bottleneck(x, int(48*scalling), (5, 5), e=3, s=1, squeeze=True, nl='HS')
    x =  _bottleneck(x, int(96*scalling), (5, 5), e=6, s=2, squeeze=True, nl='HS')
    x =  _bottleneck(x, int(96*scalling), (5, 5), e=6, s=1, squeeze=True, nl='HS')
    x =  _bottleneck(x, int(120*scalling), (5, 5), e=6, s=1, squeeze=True, nl='HS')

    x =  _conv_block(x, int(576*scalling), (1, 1), strides=(1, 1), nl='HS')
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, 1, int(576*scalling)))(x)

    x = Conv2D(int(1280*scalling), (1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x =  _return_activation(x, 'HS')
    
    x=Dropout(0.5)(x)
    xage = Conv2D(1, (1, 1), padding='same')(x)
    xgender=Conv2D(2, (1, 1), padding='same', activation='softmax')(x)
    xemo=Conv2D(6, (1, 1), padding='same', activation='softmax')(x)
    xconcate=Concatenate()([xage,xgender,xemo])
    output = Reshape((9,))(xconcate)

    model = Model(inputs, output)


    return model


In [11]:



def accu5(y_true,y_pred):
    y_true=y_true[:,0]
    y_pred=y_pred[:,0]
    

    y_pred = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))
    y_true = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))
    diff=y_true-y_pred
    abss=tf.math.abs(diff)
    zeros=tf.zeros(tf.shape(y_true),dtype=tf.dtypes.float32)
    ones=tf.ones(tf.shape(y_true),dtype=tf.dtypes.float32)
    minuss=abss-(ones*5)
    morethan=tf.clip_by_value(
    minuss,
    0,
    15,
    name=None
    )
    elements_equal_to_value = tf.equal(zeros, morethan)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    acc=count/tf.size(y_true)
    
    return acc

def meann(y_true,y_pred):
    y_true=y_true[:,0]
    y_pred=y_pred[:,0]
    

    y_pred = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))
    y_true = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))  
    aa=tf.reduce_mean(tf.abs((y_true-y_pred)))
    
    return aa



def accu10(y_true,y_pred):
    y_true=y_true[:,0]
    y_pred=y_pred[:,0]
    

    y_pred = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))
    y_true = tf.boolean_mask(y_true, tf.not_equal(y_true, -1)) 
    diff=y_true-y_pred
    abss=tf.math.abs(diff)
    zeros=tf.zeros(tf.shape(y_true),dtype=tf.dtypes.float32)
    ones=tf.ones(tf.shape(y_true),dtype=tf.dtypes.float32)
    minuss=abss-(ones*10)
    morethan=tf.clip_by_value(
    minuss,
    0,
    15,
    name=None
    )
    elements_equal_to_value = tf.equal(zeros, morethan)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    acc=count/tf.size(y_true)
    return acc

def accu15(y_true,y_pred):
    y_true=y_true[:,0]
    y_pred=y_pred[:,0]
    

    y_pred = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))
    y_true = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))
    
    diff=y_true-y_pred
    abss=tf.math.abs(diff)
    zeros=tf.zeros(tf.shape(y_true),dtype=tf.dtypes.float32)
    ones=tf.ones(tf.shape(y_true),dtype=tf.dtypes.float32)
    minuss=abss-(ones*15)
    morethan=tf.clip_by_value(
    minuss,
    0,
    15,
    name=None
    )
    elements_equal_to_value = tf.equal(zeros, morethan)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    acc=count/tf.size(y_true)
    return acc






def accu15perc(y_true,y_pred):
    y_true=y_true[:,0]
    y_pred=y_pred[:,0]
    

    y_pred = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))
    y_true = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))
    
    
    zeros=tf.zeros(tf.shape(y_true),dtype=tf.dtypes.float32)
    ones=tf.ones(tf.shape(y_true),dtype=tf.dtypes.float32)
    percentage=y_pred/y_true
    diff=percentage-ones #diff in percent
    abss=tf.math.abs(diff)

    minuss=abss-(ones*0.15)
    morethan=tf.clip_by_value(
    minuss,
    0,
    15,
    name=None
    )
    elements_equal_to_value = tf.equal(zeros, morethan)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    acc=count/tf.size(y_true)
    return acc



def accu25perc(y_true,y_pred):
    
    
    y_true=y_true[:,0]
    y_pred=y_pred[:,0]
    

    y_pred = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))
    y_true = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))

    
    zeros=tf.zeros(tf.shape(y_true),dtype=tf.dtypes.float32)
    ones=tf.ones(tf.shape(y_true),dtype=tf.dtypes.float32)
    percentage=y_pred/y_true
    diff=percentage-ones #diff in percent
    abss=tf.math.abs(diff)

    minuss=abss-(ones*0.25)
    morethan=tf.clip_by_value(
    minuss,
    0,
    15,
    name=None
    )
    elements_equal_to_value = tf.equal(zeros, morethan)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    acc=count/tf.size(y_true)
    return acc

def accu30perc(y_true,y_pred):
    
    
    y_true=y_true[:,0]
    y_pred=y_pred[:,0]
    

    y_pred = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))
    y_true = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))

    
    zeros=tf.zeros(tf.shape(y_true),dtype=tf.dtypes.float32)
    ones=tf.ones(tf.shape(y_true),dtype=tf.dtypes.float32)
    percentage=y_pred/y_true
    diff=percentage-ones #diff in percent
    abss=tf.math.abs(diff)

    minuss=abss-(ones*0.30)
    morethan=tf.clip_by_value(
    minuss,
    0,
    15,
    name=None
    )
    elements_equal_to_value = tf.equal(zeros, morethan)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    acc=count/tf.size(y_true)
    return acc

def genderAccu(y_true,y_pred):
    
    
    
    y_true=y_true[:,1]
    y_pred=y_pred[:,1:3]
    y_pred1 = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))
    y_true1 = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))
    
    return tf.keras.metrics.sparse_categorical_accuracy(y_true1, y_pred1)

def emoAccu(y_true,y_pred):
    y_true=y_true[:,2]
    y_pred=y_pred[:,3:9]
    y_pred1 = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))
    y_true1 = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true1, y_pred1)

In [12]:
def customLoss(yTrue,yPred):
    ageYTrue=yTrue[:,0]
    ageYPred=yPred[:,0]
    #print('yTrue',yTrue)
    #print('yPred',yPred)
    genderYTrue=yTrue[:,1]
    genderYPred=yPred[:,1:3]
    emoYTrue=yTrue[:,2]
    emoYPred=yPred[:,3:9]
    
    ageYTrue_masked = tf.boolean_mask(ageYTrue, tf.not_equal(ageYTrue, -1))
    ageYPred_masked = tf.boolean_mask(ageYPred, tf.not_equal(ageYTrue, -1))
    
   # print('genderYTrue',genderYTrue)
   # print('genderYPred',genderYPred)
   # print('tf.not_equal(genderYTrue, -1)',tf.not_equal(genderYTrue, -1))
    genderYTrue_masked = tf.boolean_mask(genderYTrue, tf.not_equal(genderYTrue, -1))
    genderYPred_masked = tf.boolean_mask(genderYPred, tf.not_equal(genderYTrue, -1))
    
   # print('genderYTrue_masked',genderYTrue_masked)
   # print('genderYPred_masked',genderYPred_masked)
    emoYTrue_masked = tf.boolean_mask(emoYTrue, tf.not_equal(emoYTrue, -1))
    emoYPred_masked = tf.boolean_mask(emoYPred, tf.not_equal(emoYTrue, -1))
    
    ageLoss=tf.keras.losses.logcosh(ageYTrue_masked, ageYPred_masked)
    genderLoss=tf.keras.losses.sparse_categorical_crossentropy(genderYTrue_masked,genderYPred_masked)
    emoLoss=tf.keras.losses.sparse_categorical_crossentropy(emoYTrue_masked,emoYPred_masked)
    
    loss_op=(
       K.mean(ageLoss)+K.mean(genderLoss)*12+K.mean(emoLoss)*8
    )
    
    return loss_op

In [13]:

custom_objects={'_return_activation': _return_activation,'_conv_block':_conv_block,'_bottleneck':_bottleneck,
    '_return_activation':_return_activation,'_hard_swish':_hard_swish,'_relu6':_relu6,'accu5':accu5,'accu10':accu10,'accu15':accu15,'accu15perc':accu15perc,'accu25perc':accu25perc,'accu30perc':accu30perc,'meann':meann,'ageLoss':ageLoss,'genderLoss':genderLoss,'emoLoss':emoLoss,'customLoss':customLoss,'genderAccu':genderAccu,'emoAccu':emoAccu}


#model = load_model('./models/best_weights.hdf5', custom_objects=custom_objects)
#model = load_model('./models/best_weights 14_28595.hdf5', custom_objects=custom_objects)


In [14]:
from tensorflow.keras.callbacks import *
adam=tf.keras.optimizers.Adam(lr=0.001,beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
#model.compile(loss=customLoss, optimizer=adam) 
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                              patience=4, min_lr=0.0000002)
#earlystopping=EarlyStopping(monitor='val_loss', patience=10, verbose=2)

mirrored_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
#with mirrored_strategy.scope():
model=MobileNetV3_Large((112,112,3),9,1.5)
model.compile(loss=customLoss, optimizer=adam,metrics=[accu5,accu10,accu15,accu15perc,accu25perc,accu30perc,meann,ageLoss,genderLoss,emoLoss,emoAccu,genderAccu]) 


checkpointer = ModelCheckpoint(filepath="./models/best_weights.hdf5", 
                               monitor = 'val_loss',
                               mode='min',
                               verbose=1, 
                               save_best_only=True)
tensorboard=TensorBoard(log_dir='./logs')
callbackss=[checkpointer,tensorboard]

INFO:tensorflow:Single-worker CollectiveAllReduceStrategy with local_devices = ('/device:GPU:0',), communication = CollectiveCommunication.AUTO


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,  img_to_array, load_img




In [16]:
#### !!!!!!!!!!!! fIX flip

import gc
from random import randrange

#generator = ImageDataGenerator_landmarks()
#valix, valiy = validation_generator
#image, y = training_generator

image=inputarraytrain
y=labelarraytrain

valix=inputarraytest
valiy=labelarraytest

#y=listimagewithlabel


#with tf.device("device:XLA_GPU:1"):
for i in range (1000):
    if (i !=0):
        del datagen
    gc.collect()
    datagen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    zoom_range=[0.8,1.2]
    )

    i11=i%6
    print('i:',i)
    traincnt=int(inputarraytrain.shape[0]/6)-1
    valicnt=int(inputarraytest.shape[0]/4)-1

    imageportion=inputarraytrain[traincnt*i11:traincnt*(i11+1)]
    labelportion=labelarraytrain[traincnt*i11:traincnt*(i11+1)]
    listk=np.random.choice(range(inputarraytest.shape[0]), inputarraytest.shape[0]//4, replace=False)

    testimageportion=inputarraytest[listk]
    testlabelportion=labelarraytest[listk]
    print(imageportion.shape)

    #print('labelportion[0]:',labelportion[0])



    train_history = model.fit_generator(

                        datagen.flow(imageportion, labelportion, batch_size=250),
                        #steps_per_epoch=500,
                        steps_per_epoch=int(imageportion.shape[0]/250)+1,
        validation_data=(testimageportion, testlabelportion),
        epochs=1,
                        max_queue_size=2,
                        workers = 2 ,
                        use_multiprocessing=True,
                        #validation_steps=60,
                        callbacks = callbackss)




"""

history = model.fit(x=image,y=y,
                    batch_size=300,
                    #steps_per_epoch=900,
                    epochs=500,validation_data=(valix, valiy),
                    validation_steps=60
                    )
"""

i: 0
(67971, 112, 112, 3)
Instructions for updating:
Use tf.identity instead.
 44/272 [===>..........................] - ETA: 2:32 - loss: 54.0168 - accu5: 0.0168 - accu10: 0.0452 - accu15: 0.0865 - accu15perc: 0.0112 - accu25perc: 0.0213 - accu30perc: 0.0264 - meann: 33.5737 - ageLoss: 32.8817 - genderLoss: 8.9405 - emoLoss: 12.1946 - emoAccu: 0.4815 - genderAccu: 0.5940

Process Keras_worker_ForkPoolWorker-1:
Process Keras_worker_ForkPoolWorker-2:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)

KeyboardInterrupt: 

In [ ]:
    train_history = model.fit_generator(

                        datagen.flow(imageportion, labelportion, batch_size=250),
                        #steps_per_epoch=500,
                        steps_per_epoch=int(imageportion.shape[0]/250)+1,
        validation_data=(testimageportion, testlabelportion),
        epochs=1,
                        max_queue_size=2,
                        workers = 2 ,
                        use_multiprocessing=True,
                        #validation_steps=60,
                        callbacks = callbackss)

In [ ]:

    train_history = model.fit(

                        datagen.flow(imageportion, labelportion, batch_size=250),
                        #steps_per_epoch=500,
                        steps_per_epoch=int(imageportion.shape[0]/250)+1,
        validation_data=(testimageportion, testlabelportion),
        epochs=1,
        shuffle=False,
                       # max_queue_size=2,
                      #  workers = 2 ,
                     #   use_multiprocessing=True,
                        #validation_steps=60,
                        callbacks = callbackss)

In [ ]:
print(inputarraytrain.shape)

In [ ]:
%reset_selective -f labelarraytrain
%reset

In [ ]:
import os
import random

fromtestarray=0
onlyWrongOnes=1
#path ='./fd2/img_celeba_resize/'
path ='./'
files = os.listdir(path)
index = random.randrange(0, len(files))

wronglist=[]
from tensorflow.keras.preprocessing import image as imakeras
for i in range (inputarraytest.shape[0]):
    
    #print(inputarraytest.shape)
    #print(labelarraytest.shape)
    if(fromtestarray==1):
        
        #index = random.randrange(0,inputarraytest.shape[0] )
        index = i
        img = inputarraytest[index]
        img_tensor = imakeras.img_to_array(img)
    else:
        index = random.randrange(0, len(files))
    
        img = imakeras.load_img(path+'11w2.png', target_size=(112, 112))
        img_tensor = imakeras.img_to_array(img)
        img_tensor = img_tensor[:,:,::-1]
    #img = imakeras.load_img('./geg.jpg', target_size=(40, 40))
    #print(img)

    
    img_tensor = np.expand_dims(img_tensor, axis=0)

    

    img_tensor=np.concatenate((img_tensor,img_tensor))#2
    img_tensor=np.concatenate((img_tensor,img_tensor))#4
    img_tensor=np.concatenate((img_tensor,img_tensor))#8
  #  img_tensor=np.concatenate((img_tensor,img_tensor))#16
  #  img_tensor=np.concatenate((img_tensor,img_tensor))#32
  #  img_tensor=np.concatenate((img_tensor,img_tensor))#64
  #  img_tensor=np.concatenate((img_tensor,img_tensor))#128
    start_time = time.time()
    aa=model.predict(img_tensor,steps=1)
    #print('aa:',aa)
    elapsed_time = time.time() - start_time
    aa=aa[0]
    label=labelarraytest[index]
    #print('argmax(aa',np.argmax(aa))
    #print(aa.shape)

    #if(np.argmax(aa)!=5):
        #continue
    
    #print(aa[0])
   # if(aa-label<8 and aa-label>-8):
     #   continue

    
    print("ela"+str(elapsed_time))

    print('index: ', index)
    print('pd: ',aa)

    print('label: ',label)
   # print('name: ',namearraytest[index])
   # wronglist.append(namearraytest[index])
    if(fromtestarray==1):
        img = img[:,:,::-1]
   # print(img.shape)
    plt.imshow(img)
    plt.show()
    

In [ ]:
#model = load_model('./models/best_weights.hdf5',custom_objects={'customLoss': customLoss})


In [ ]:
"""
model = load_model('./models/kerasmodelSmaller/best_weights.hdf5',custom_objects={'customLoss': customLoss})
from kerasulti import *
from tensorflow.core.framework import attr_value_pb2
from tensorflow.core.framework import graph_pb2
from tensorflow.core.framework import node_def_pb2
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_util
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util import deprecation
from tensorflow.python.util.tf_export import tf_export
from tensorflow.python.tools import import_pb_to_tensorboard
#convert hdf5 to pb
input_path = './models/kerasmodelSmall/'
weight_file = 'best_weights.hdf5'
weight_file_path = osp.join(input_path,weight_file)
output_graph_name = weight_file[:-5] + '.pb'
#轉換函式
def h5_to_pb(h5_model,output_dir,model_name,out_prefix = "output_",log_tensorboard = True):
    if osp.exists(output_dir) == False:
        os.mkdir(output_dir)
    out_nodes = []
    for i in range(len(h5_model.outputs)):
        out_nodes.append(out_prefix + str(i + 1))
        tf.identity(h5_model.output[i],out_prefix + str(i + 1))
    sess = K.get_session()
    from tensorflow.python.framework import graph_util,graph_io
    init_graph = sess.graph.as_graph_def()
    main_graph = convert_variables_to_constants(sess,init_graph,out_nodes)
    graph_io.write_graph(main_graph,output_dir,name = model_name,as_text = False)
    if log_tensorboard:

        import_pb_to_tensorboard.import_to_tensorboard(osp.join(output_dir,model_name),output_dir)
#輸出路徑
output_dir = osp.join(os.getcwd(),"trans_model")
#載入模型
h5_model = load_model(weight_file_path, custom_objects={'customLoss': customLoss})
h5_to_pb(h5_model,output_dir = output_dir,model_name = output_graph_name)
print('model saved')
"""

In [ ]:

#the pb frozen with following code can be converted to tflite
import os 
import os.path as osp
from tensorflow.keras import backend as K
from tensorflow.core.framework import attr_value_pb2
from tensorflow.core.framework import graph_pb2
from tensorflow.core.framework import node_def_pb2
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_util
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util import deprecation
from tensorflow.python.util.tf_export import tf_export

K.set_learning_phase(0)

model = load_model('./models/best_weights.hdf5',custom_objects= customLoss)

print(model.outputs)
# [<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]
print(model.inputs)
# [<tf.Tensor 'conv2d_1_input:0' shape=(?, 28, 28, 1) dtype=float32>]

In [ ]:
from tensorflow.python.tools import import_pb_to_tensorboard

In [ ]:
print(model.outputs)
# [<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]
print(model.inputs)

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

In [ ]:

# Save to ./model/tf_model.pb
tf.train.write_graph(frozen_graph, "trans_model/", "tf_model.pb", as_text=False)

In [ ]:
converter = tf.lite.TFLiteConverter.from_frozen_graph('trans_model/tf_model.pb', ['input_1'], ['dense_1/BiasAdd'])
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

In [ ]:
from tensorflow.python.tools import import_pb_to_tensorboard
import_pb_to_tensorboard.import_to_tensorboard('trans_model/small/tf_model.pb','/workspace/testbuildmodel/lanes/trans_model/small')

In [ ]:
print(valix)
print(valiy)


In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')